# **SMS SPAM Classification**

**1. Import required library**

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras.utils import to_categorical
from keras.models import load_model

In [2]:
import csv
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')  
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**2. Read dataset and do pre-processing**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('/content/drive/MyDrive/spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
df.drop(['Unnamed: 2','Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [6]:
df.groupby(['v1']).size()

v1
ham     4825
spam     747
dtype: int64

In [7]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [8]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [9]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

**3. Create Model**


**4. Add Layers (LSTM, Dense-(Hidden Layers), Output)**

In [10]:
inputs = Input(name='InputLayer',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

**5. Compile the Model**

In [11]:
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FullyConnectedLayer1 (Dense  (None, 256)              16640     
 )                                                               
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                             

**6. Fit the Model**

In [12]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=20,validation_split=0.2)

Epoch 1/20
30/30 [==============================] - 11s 256ms/step - loss: 0.3462 - accuracy: 0.8762 - val_loss: 0.1576 - val_accuracy: 0.9346
Epoch 2/20
30/30 [==============================] - 7s 237ms/step - loss: 0.0907 - accuracy: 0.9784 - val_loss: 0.0461 - val_accuracy: 0.9863
Epoch 3/20
30/30 [==============================] - 7s 235ms/step - loss: 0.0495 - accuracy: 0.9865 - val_loss: 0.0406 - val_accuracy: 0.9852
Epoch 4/20
30/30 [==============================] - 8s 285ms/step - loss: 0.0319 - accuracy: 0.9908 - val_loss: 0.0461 - val_accuracy: 0.9810
Epoch 5/20
30/30 [==============================] - 7s 233ms/step - loss: 0.0342 - accuracy: 0.9916 - val_loss: 0.0419 - val_accuracy: 0.9852
Epoch 6/20
30/30 [==============================] - 7s 234ms/step - loss: 0.0189 - accuracy: 0.9947 - val_loss: 0.0416 - val_accuracy: 0.9842
Epoch 7/20
30/30 [==============================] - 7s 235ms/step - loss: 0.0168 - accuracy: 0.9947 - val_loss: 0.0451 - val_accuracy: 0.9863
Epoch

**7. Save The Model**

In [13]:
model.save("Ai_Spam_Identifier")

**8. Test The Model**

In [14]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [15]:
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.3f}'.format(accuracy[1]))

27/27 [==============================] - 1s 20ms/step - loss: 0.2540 - accuracy: 0.9737
Accuracy: 0.974


In [16]:
y_pred = model.predict(test_sequences_matrix)
print(y_pred[25:40].round(3))

27/27 [==============================] - 1s 19ms/step
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]]


In [17]:
print(Y_test[25:40])

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
